# 16 1 5

Adding in expected secondary position, which will be used to find the observed secondary lobe closest to the expected secondary position. 

PS3 takes in the following data fields: 
- *TID* — Target ID. If the Target ID is a “D”, then PS3 will process this file, otherwise it will be ignored.
This allows a master database to contain, for instance, single stars used for deconvolution, slit mask
or drift calibrations, etc.
- *DSFN* — Double star (FITS cube) file name.
- *RSFN* — Reference star (FITS) cube file name.
- *ThetaC* — Calculated (input) estimated, predicted, or last catalog reported double star position angle.
- *RhoC* — Calculated (input) estimated, predicted, or last catalog reported double star separation.
- *Delta* — Camera Angle (degrees). Camera orientation angle with respect to the sky.
- *E* — Plate scale (arc seconds / pixel).

For all the FITS data I am working with from April 2014 Kitt Peak run:
- *Delta* = 0 [degrees]
- *E* = 0.01166 [arc seconds / pixel]


In [1]:
# Using calculated centroid data from end of previous notebook along 
#  with calibration and expected position data from run log to calculate 
#  observed secondary location.

In [2]:
# Import Modules
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.fftpack import fft2,ifft2,fftshift
from speckle_fns import fits_import, circ_filter1
from speckle_fns import deconv0,deconv1, postprocess
from speckle_fns import filter_lpf, filter_hpf, filter_interference
from speckle_fns import fits_view
from collections import namedtuple
import sys
import cv2
import math

%matplotlib inline

In [3]:
# Centroid values from end of 15-12-28 notebook:
centroids = np.zeros((5,2,2))

centroids[0] = [[ 403.24772626, 405.91946966],[ 108.75227374, 106.08053034]]
centroids[1] = [[ 220.00967484, 255.04554408],[ 291.99032516, 256.95445592]]
centroids[2] = [[ 469.02557551, 506.57034716],[ 42.97582522, 5.06646098]]
centroids[3] = [[ 225.11283002, 274.88879163],[ 286.88716998, 237.11120837]]
centroids[4] = [[ 282.11470187, 275.93283984],[ 229.88529813, 236.06716016]]

# Expected positions of each pair from "KP 4-14 Run Log.xlsx"
thetaCs = np.array((204,63,32,305,124))
rhoCs = np.array((44.7,4.9,0.3,0.5,0.4))

# Calibration values from "KP 4-14 Run Log.xlsx"
delta = 0
E = 0.01166

In [ ]:
# Select which set of data to work with

k = 3

centroid = centroids[k]

In [4]:
# Functions & structs used in this file

# convert float pixel indices to rounded, uint8 values
def pixelRound(val):
    return (np.round(val)).astype(np.int16)

# convert theta and rho values to image x and y coordinates
def polar2cart(rho, theta, imgSize):
    midpoint = imgSize/2 # Centered on midpoint of image
    x = midpoint + (rho * np.cos(np.radians(theta-90)))
    # Subtraction because positive direction of image is down on screen
    y = midpoint - (rho * np.sin(np.radians(theta-90)))
    
    return (x,y)

# posType struct is for holding the position of anything
#  Position may be represented in Cartesian or Polar coordinates
#  Want to be able to store both kinda of coordinates
class posType:
    def __init__(self, midpoint=None, theta=None, rho=None, x=None, y=None):
        self.theta = theta
        self.rho = rho
        self.x = x
        self.y = y
        self.midpoint = midpoint
        
    # Orbital plot has 0deg pointing down on the screen [resulting in 90deg shift]
    # Theta rotates counterclockwise from 0deg
    # Positive pixel direction is downward [resulting in Y phase reversal]
    # Polar coordinates centered on middle pixel of image
        
    # convert theta and rho values to image x and y coordinates    
    def polar2cart(self):
        self.x = self.midpoint + (self.rho * np.cos(np.radians(self.theta-90)))
        self.y = self.midpoint - (self.rho * np.sin(np.radians(self.theta-90)))
        
    # convert x and y coordinates to theta and rho values
    def cart2polar(self):
        # Get cartesians centered on 0 for calculations
        x = self.x-self.midpoint
        y = self.midpoint-self.y
        
        self.rho = np.linalg.norm((x,y))
        self.theta = np.degrees(np.arctan(y/x))+90
        # Add 180 deg if in quadrants 2 or 3
        self.theta = (x<0)*180 + self.theta
        
# starType struct is for holding position of a star
#  A star has coordinates in camera (in pixels) and also
#  in the observed sky (in arcseconds)
#starType = namedtuple('starType',"cam sky error")
class starType:
    def __init__(self, mid = None, error=None, E=None, delta=None):
        self.cam = posType(midpoint = mid)
        self.sky = posType(midpoint = mid)
        self.error = error
        self.E = E
        self.delta = delta


In [ ]:
## Displaying calculated centroid values along with expected secondary star locations

# Create blank image
imgDim = 512
img = np.zeros((imgDim,imgDim,3), np.uint8)

# Object for the expected star position
expectedPos = starType(mid = imgDim/2)

# Assigning position values from previous data
expectedPos.sky.theta = thetaCs[k]
expectedPos.sky.rho = rhoCs[k]

# Calculate camera locations of expected star pos
expectedPos.cam.theta = expectedPos.sky.theta-delta
expectedPos.cam.rho = expectedPos.sky.rho/E
# Converting to cartesian
expectedPos.cam.polar2cart()

# Parameters for marking objects
circle_radius = 11 # Radius of circle around location
circle_border = 1 # Border of circle around location
color_centroid = (255,0,0) # Color of centroid marker
color_secondaryC = (0,255,0) # Color of expected secondary star location

# Mark centroid locations on image
cv2.circle(img, (centroid[0,0].astype(np.uint16),centroid[0,1].astype(np.uint16)), circle_radius, color_centroid, circle_border)
cv2.circle(img, (centroid[1,0].astype(np.uint16),centroid[1,1].astype(np.uint16)), circle_radius, color_centroid, circle_border)

# Mark expected star location on image
cv2.circle(img, (pixelRound(expectedPos.cam.x),pixelRound(expectedPos.cam.y)),circle_radius, color_secondaryC, circle_border)

plt.figure(figsize=(10,10))
plt.imshow(img)

In [8]:

expectedPos = starType(mid = 512/2)

expectedPos.cam.rho = 50
expectedPos.cam.theta = 0
print("Polar Coords:")
print(expectedPos.cam.rho, expectedPos.cam.theta)

print("Converting to Cartesian")
expectedPos.cam.polar2cart()

print("Cartesian Coords:")
print(expectedPos.cam.x, expectedPos.cam.y)

print("Converting to Polar")
expectedPos.cam.cart2polar()

print("Polar Coords:")
print(expectedPos.cam.rho, expectedPos.cam.theta)

Polar Coords:
50 0
Converting to Cartesian
Cartesian Coords:
256.0 306.0
Converting to Polar
Polar Coords:
50.0 0.0


/home/niels/anaconda3/envs/speckle/lib/python3.5/site-packages/ipykernel/__main__.py:44: RuntimeWarning: divide by zero encountered in double_scalars
